In [ ]:
#Apartado 2

import cv2
import numpy as np

file_path = 'tráfico01.mp4'
video = cv2.VideoCapture(file_path)

FOI = video.get(cv2.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=30)

frames = []
for frameOI in FOI:
    video.set(cv2.CAP_PROP_POS_FRAMES, frameOI)
    ret, frame = video.read()
    frame = cv2.resize(frame, (1600, 900))
    frames.append(frame)

result2 = np.median(frames, axis=0).astype(dtype=np.uint8)
cv2.imwrite('sinCoches.jpg', result2)

In [16]:
#Apartado 3
import cv2 as cv
import numpy as np

# capture frames from a camera
cap = cv.VideoCapture("tráfico01.mp4")
# creamos un substractor de fondo
subf = cv.bgsegm.createBackgroundSubtractorMOG()
kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3,3))
# Creamos un contador de coches
Contador_de_Coches = 0

while True:
    # leémos los frames del vídeo
    ret, frame = cap.read()
    if ret == False: break
    # Creamos una copia de una parcela del frame dónde detectaremos los coches
    Parcela = frame[367:915, 943:1623]
    # Aplicamos el filtro gaussiano para remover el ruido
    frame = cv.GaussianBlur(frame, (7,7), 0)
    # Aplicamos el substractor de fondo creado previamente a la parcela para operar con él
    fgmask = subf.apply(Parcela)
    fgmask = cv.morphologyEx(fgmask, cv.MORPH_OPEN, kernel)
    # Expandimos el frame
    fgmask = cv.dilate(fgmask, None, iterations=2)
    # Hallamos los contornos para detectar los coches
    contornos,_ = cv.findContours(fgmask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    # Dibujamos una línea en la parcela, que nos ayudará a contar los coches
    línea = cv.line(Parcela, (21, 398), (1500, 398), (255,0,0), 1)
    for contorno in contornos:
        area = cv.contourArea(contorno)
        if area > 1800:
            x, y, w, h = cv.boundingRect(contorno)
            # Creamos un área que marcará a cada coche
            cv.rectangle(Parcela, (x,y), (x+w, y+h), (0,255,0), 1)
            # Contamos los coches que atraviesen la línea previamente dibujada
            if (398 - 7)< y <(398 + 7):
                Contador_de_Coches = Contador_de_Coches + 1
    # Creamos un rótulo que nos muestre el estado del contrador de coches
    cv.putText(frame, 'Contador_de_Coches = ' + str(Contador_de_Coches), (602,142), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 255), 3)
    # Seleccionamos la parcela sobre la que hemos efectuado la detección de coches y la sustituimos por la del frame original
    frame[367:915, 943:1623] = Parcela
    cv.imshow('frame', cv.resize(frame, None, fx= .5, fy= .5))
    #cv.imshow('fmask', fgmask)
    k = cv.waitKey(10) & 0xFF
    if k == 27:
        break

cap.release()
cv.destroyAllWindows()